In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import date

# The plot module for graphs
import matplotlib.pyplot as plt
%matplotlib inline

# The statistical module for statistical analysis
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

In [29]:
# File to Load (Remember to Change These)
weather_data_to_load = "clean_weather_1-1-2016_to_8-31-2020.csv"
crime_data_to_load = "crime_cleaned.csv"

# Read School and Student Data File and store into Pandas DataFrames
weather_data = pd.read_csv(weather_data_to_load)
crime_data = pd.read_csv(crime_data_to_load,usecols = ['Occurrence_date','OFFENSE_TYPE_ID','OFFENSE_CATEGORY_ID','INCIDENT_ID'])
crime_data_for_pivot = pd.read_csv(crime_data_to_load,usecols = ['Occurrence_date','OFFENSE_TYPE_ID','INCIDENT_ID'])

In [30]:
weather_data['weather_date'] = pd.to_datetime(weather_data['Date'])
print(len(weather_data))
weather_data.info()

1705
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1705 entries, 0 to 1704
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Unnamed: 0           1705 non-null   int64         
 1   Date                 1705 non-null   object        
 2   Avg temperature (F)  1705 non-null   int64         
 3   Precip Inches        1705 non-null   float64       
 4   Cloud Cover          1705 non-null   float64       
 5   Wind Speed(Miles)    1705 non-null   float64       
 6   weather_date         1705 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 93.4+ KB


In [51]:
crime_data['crime_date'] = pd.to_datetime(crime_data['Occurrence_date'])

crime_data_type_only = crime_data.groupby(['crime_date', 'OFFENSE_CATEGORY_ID']).nunique()
crime_data_type_only.drop(['crime_date', 'OFFENSE_CATEGORY_ID','OFFENSE_TYPE_ID'], axis=1, inplace=True)
crime_data_type_only.reset_index()

,crime_date,OFFENSE_CATEGORY_ID,Occurrence_date,INCIDENT_ID
0,2015-01-02,aggravated-assault,1,2
1,2015-01-02,all-other-crimes,1,4
2,2015-01-02,auto-theft,1,2
3,2015-01-02,burglary,1,3
4,2015-01-02,drug-alcohol,1,3
...,...,...,...,...
26304,2020-12-07,burglary,1,1
26305,2020-12-07,drug-alcohol,1,1
26306,2020-12-07,other-crimes-against-persons,1,1
26307,2020-12-07,public-disorder,1,1


In [82]:
crime_data_pivot = crime_data_type_only.pivot_table(index='crime_date', columns = 'OFFENSE_CATEGORY_ID', values ='INCIDENT_ID')
crime_data_pivot = crime_data_pivot.fillna(0)
crime_data_pivot["total_offenses"] = crime_data_pivot.iloc[:,:].sum(axis=1)
crime_data_pivot.info()
crime_data_pivot.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2167 entries, 2015-01-02 to 2020-12-07
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   aggravated-assault            2167 non-null   float64
 1   all-other-crimes              2167 non-null   float64
 2   arson                         2167 non-null   float64
 3   auto-theft                    2167 non-null   float64
 4   burglary                      2167 non-null   float64
 5   drug-alcohol                  2167 non-null   float64
 6   larceny                       2167 non-null   float64
 7   murder                        2167 non-null   float64
 8   other-crimes-against-persons  2167 non-null   float64
 9   public-disorder               2167 non-null   float64
 10  robbery                       2167 non-null   float64
 11  sexual-assault                2167 non-null   float64
 12  theft-from-motor-vehicle      2167 non-null 

OFFENSE_CATEGORY_ID,aggravated-assault,all-other-crimes,arson,auto-theft,burglary,drug-alcohol,larceny,murder,other-crimes-against-persons,public-disorder,robbery,sexual-assault,theft-from-motor-vehicle,white-collar-crime,total_offenses
crime_date,,,,,,,,,,,,,,,
2015-01-02,2.0,4.0,0.0,2.0,3.0,3.0,5.0,0.0,2.0,4.0,2.0,0.0,2.0,1.0,30.0
2015-01-03,3.0,5.0,0.0,1.0,3.0,2.0,4.0,0.0,2.0,3.0,1.0,1.0,2.0,1.0,28.0
2015-01-04,1.0,5.0,0.0,1.0,2.0,1.0,3.0,1.0,2.0,4.0,1.0,1.0,2.0,1.0,25.0
2015-01-05,3.0,6.0,0.0,1.0,4.0,3.0,3.0,0.0,2.0,3.0,1.0,1.0,2.0,2.0,31.0
2015-01-06,2.0,5.0,0.0,1.0,3.0,3.0,3.0,0.0,2.0,4.0,2.0,1.0,2.0,3.0,31.0


In [93]:
# Combine the data into a single dataset.  
#inner merge intentionally drops days that do not match in both data sets to avoid NaN records
criminal_weather_merged = pd.merge(crime_data_pivot, weather_data, left_on = 'crime_date',right_on = 'weather_date')
criminal_weather_merged = criminal_weather_merged[['Date','total_offenses','aggravated-assault', \
                                                             'all-other-crimes','arson', 'auto-theft','burglary',\
                                                             'murder','other-crimes-against-persons', 'public-disorder',\
                                                             'robbery','sexual-assault', 'theft-from-motor-vehicle',\
                                                             'white-collar-crime','Avg temperature (F)','Precip Inches',\
                                                             'Cloud Cover', 'Wind Speed(Miles)',]]
print(criminal_weather_merged['Date'].min())
print(criminal_weather_merged['Date'].max())
criminal_weather_merged

2016-01-01
2020-08-31


,Date,total_offenses,aggravated-assault,all-other-crimes,arson,auto-theft,burglary,murder,other-crimes-against-persons,public-disorder,robbery,sexual-assault,theft-from-motor-vehicle,white-collar-crime,Avg temperature (F),Precip Inches,Cloud Cover,Wind Speed(Miles)
0,2016-01-01,38.0,3.0,7.0,0.0,2.0,3.0,1.0,3.0,4.0,1.0,2.0,2.0,3.0,11,0.0,2.000,3.375
1,2016-01-02,29.0,1.0,5.0,0.0,1.0,4.0,0.0,2.0,5.0,2.0,0.0,2.0,0.0,20,0.0,10.875,4.500
2,2016-01-03,33.0,3.0,7.0,0.0,1.0,3.0,0.0,3.0,5.0,2.0,2.0,2.0,0.0,24,0.0,10.750,3.500
3,2016-01-04,29.0,1.0,6.0,0.0,1.0,3.0,0.0,1.0,5.0,1.0,1.0,2.0,1.0,22,0.0,56.250,5.750
4,2016-01-05,33.0,2.0,6.0,0.0,1.0,4.0,0.0,3.0,3.0,2.0,0.0,2.0,1.0,28,0.0,19.750,3.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1700,2020-08-27,30.0,1.0,5.0,0.0,1.0,5.0,0.0,2.0,4.0,1.0,1.0,2.0,1.0,85,0.0,34.750,5.750
1701,2020-08-28,31.0,3.0,5.0,0.0,2.0,2.0,1.0,2.0,4.0,3.0,1.0,2.0,2.0,72,0.1,48.250,6.875
1702,2020-08-29,35.0,3.0,4.0,1.0,1.0,3.0,0.0,3.0,5.0,2.0,3.0,2.0,2.0,75,0.0,26.875,4.875
1703,2020-08-30,30.0,3.0,6.0,1.0,1.0,4.0,0.0,2.0,6.0,1.0,0.0,2.0,0.0,81,0.0,18.000,7.125
